In [1]:
import pandas as pd
import numpy as np
import json

In [2]:

def generate_info(dataid, name, type, subtype, subtype2, rate, size, uint,
                  notes):
    data = {
        name: {
            "data_id": dataid,
            "data_name": name,
            "data_type": type,
            "data_subtype1": subtype,
            "data_subtype2": subtype2,
            "data_rate": rate,
            "data_size": size,
            "data_unit": uint,
            "data_notes": notes
        }
    }
    return data


In [3]:
data = pd.read_csv("Failures Modes in MCVT v6 - Failure Modes of MCVT.csv")

data['shape'] = data.groupby('Component ID').transform('count')['System']

data['Component Fault'] = data['Component Fault'].astype(str)

data['fault'] = data.groupby('Component ID').transform(lambda x: ','.join(x))['Component Fault']

/var/folders/jg/vffvcnyd6_bfssm4fcwph6b00000gn/T/ipykernel_65373/3036708812.py:7: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the function.
  data['fault'] = data.groupby('Component ID').transform(lambda x: ','.join(x))['Component Fault']
/var/folders/jg/vffvcnyd6_bfssm4fcwph6b00000gn/T/ipykernel_65373/3036708812.py:7: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the function.
  data['fault'] = data.groupby('Component ID').transform(lambda x: ','.join(x))['Component Fault']
/var/folders/jg/vffvcnyd6_bfssm4fcwph6b00000gn/T/ipykernel_65373/3036708812.py:7: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeEr

In [5]:
trans = {
    'ECLSS': 4,
    'Structure': 2,
    'SPL' : 8,
    'Disturbance' : 9,
    'Interior Environment' : 6,
    'Power': 3,
}

In [6]:
def _sensor_id(x):
    return (x // 1000) * 100 + x % 100

In [7]:
result = {}
count = 0

for _, row in data.iterrows():
    count += 1
    dataid = row['Component ID']
    name = row['System/Sub-system/Component']
    dtype = 4 if "Sensor" in row['fault'] else 3
    subtype = trans[row['System']]
    subtype2 = 255
    rate = 1
    size = row['shape']
    unit = 'n/a'
    notes = row['fault']
    result.update(generate_info(dataid, name, dtype, subtype, subtype2, rate, size, unit, notes))


    if row['Activity Type'] == 3:
        dataid = _sensor_id(row['Component ID'])
        name = row['System/Sub-system/Component'] + "_Measure"
        dtype = 1 
        subtype = trans[row['System']]
        subtype2 = 255
        rate = 1
        size = 1
        unit = 'n/a'
        notes = row['Component']b
        result.update(generate_info(dataid, name, dtype, subtype, subtype2, rate, size, unit, notes))

In [23]:
data['System'].unique()

array(['ECLSS', 'Structure', 'SPL', 'Disturbance', 'Interior Environment',
       'Power'], dtype=object)

In [63]:
json_string = json.dumps(result)
with open('db_info_v6.json', 'w') as outfile:
    outfile.write(json_string)